In [11]:
import tensorflow as tf

import numpy as np
import os
import time

## Read the Data

In [12]:
path_to_file = 'PGW.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [13]:
print('Length of text: {} characters'.format(len(text)))

Length of text: 7743140 characters


In [14]:
print(text[:250])



CHAPTER

  1.  MR. JACKSON MAKES UP HIS MIND
  2.  SEDLEIGH
  3.  PSMITH
  4.  STAKING OUT A CLAIM
  5.  GUERRILLA WARFARE
  6.  UNPLEASANTNESS IN THE SMALL HOURS
  7.  ADAIR
  8.  MIKE FINDS OCCUPATION
  9.  THE FIRE BRIGADE MEETING
 10.  ACHILLES


In [15]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

99 unique characters


## Process the data

In [16]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '$' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [10]:
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\nCHAPTER\n\n  ' ---- characters mapped to int ---- > [ 0  0 33 38 31 46 50 35 48  0  0  1  1]


In [17]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])





C
H
A


In [18]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\n\nCHAPTER\n\n  1.  MR. JACKSON MAKES UP HIS MIND\n  2.  SEDLEIGH\n  3.  PSMITH\n  4.  STAKING OUT A CLAIM\n'
'  5.  GUERRILLA WARFARE\n  6.  UNPLEASANTNESS IN THE SMALL HOURS\n  7.  ADAIR\n  8.  MIKE FINDS OCCUPATI'
"ON\n  9.  THE FIRE BRIGADE MEETING\n 10.  ACHILLES LEAVES HIS TENT\n 11.  THE MATCH WITH DOWNING'S\n 12. "
' THE SINGULAR BEHAVIOR OF JELLICOE\n 13.  JELLICOE GOES ON THE SICK LIST\n 14.  MIKE RECEIVES A COMMISS'
'ION\n 15.  ... AND FULFILLS IT\n 16.  PURSUIT\n 17.  THE DECORATION OF SAMMY\n 18.  MR. DOWNING ON THE SC'


In [19]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nCHAPTER\n\n  1.  MR. JACKSON MAKES UP HIS MIND\n  2.  SEDLEIGH\n  3.  PSMITH\n  4.  STAKING OUT A CLAIM'
Target data: '\nCHAPTER\n\n  1.  MR. JACKSON MAKES UP HIS MIND\n  2.  SEDLEIGH\n  3.  PSMITH\n  4.  STAKING OUT A CLAIM\n'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 33 ('C')
Step    2
  input: 33 ('C')
  expected output: 38 ('H')
Step    3
  input: 38 ('H')
  expected output: 31 ('A')
Step    4
  input: 31 ('A')
  expected output: 46 ('P')


In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build the model

In [21]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 99) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           25344     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 99)            101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([55, 83, 84, 60, 20, 32, 24, 49,  7, 23, 76, 51, 90, 56, 74, 62, 88,
       19, 15, 87, 72, 68, 93,  1, 24, 35, 48, 27, 62,  8,  5, 91, 80,  1,
       89, 42, 79, 41, 77, 31, 31, 54, 85, 56, 25, 13, 81, 89, 35, 11, 85,
       77, 37, 17, 80, 27, 62, 87,  5, 85, 39, 74, 24, 37, 88, 24, 79,  6,
       80, 37, 46, 77, 42,  2, 68, 17, 36, 74, 29,  4, 71, 52, 75, 55, 38,
       52, 66, 16, 59, 93, 18, 92, 17, 36, 12, 56, 77,  4, 35, 70])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'cinates me. A grand pursuit, sir."\n\n"Undoubtedly, Smith. I am very pleased, very pleased indeed. I w'

Next Char Predictions: 
 "Yxya4B8S'7qUáZoc§3/£miè 8ER;c(%âu àLtKrAAXzZ9-vàE+zrG1u;c£%zIo8G§8t&uGPrL!i1Fo?$lVpYHVg0_è2æ1F,Zr$Ek"


## Train the model

In [31]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 99)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.595563


In [34]:
model.compile(optimizer='adam', loss=loss)

In [4]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1197/1197 [==============================] - 1507s 1s/step - loss: 1.7645
Epoch 2/10
1197/1197 [==============================] - 1526s 1s/step - loss: 1.3241
Epoch 3/10
1197/1197 [==============================] - 3759s 3s/step - loss: 1.2548
Epoch 4/10
1197/1197 [==============================] - 3156s 3s/step - loss: 1.2182
Epoch 5/10
1197/1197 [==============================] - 1783s 1s/step - loss: 1.1943
Epoch 6/10
1197/1197 [==============================] - 1467s 1s/step - loss: 1.1766
Epoch 7/10
1197/1197 [==============================] - 1633s 1s/step - loss: 1.1636
Epoch 8/10
1197/1197 [==============================] - 1985s 2s/step - loss: 1.1538
Epoch 9/10
1197/1197 [==============================] - 1579s 1s/step - loss: 1.1466
Epoch 10/10
1197/1197 [==============================] - 1561s 1s/step - loss: 1.1421


## Generate text

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            25344     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 99)             101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 350000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [32]:
# testing the initial training, with .5 temperature and 10k characters

# print(generate_text(model, start_string=u"It"))

In [33]:
# now 350,000 characters to approximate 50k words
novel = generate_text(model, start_string=u'It')

In [34]:
novel_one = open('novels/novel_one.txt', 'w')
novel_one.write(novel)
novel_one.close()